# Домашнее задание к семинару 07 (HW07)

Тема: кластеризация, внутренние метрики качества, PCA/t-SNE и "честный" unsupervised-эксперимент на синтетических данных.

HW07 относится к семинару **S07** и выполняется в личном репозитории студента (на основе шаблона курса) в папке `homeworks/HW07/`.

---

## 1. Цель

Закрепить:

- понимание различий между семействами методов кластеризации:
  - **KMeans** (геометрия "шаров" и выбор `k`),
  - **DBSCAN** (плотность, шум, выбросы, нелинейные формы),
  - **Agglomerative** (иерархическая логика и влияние `linkage`);
- навыки корректного **препроцессинга** для distance-based методов:
  - масштабирование,
  - обработка пропусков,
  - (при необходимости) кодирование категориальных признаков;
- оценку качества кластеризации **без истинных меток**:
  - `silhouette_score` (выше – лучше),
  - `davies_bouldin_score` (ниже – лучше),
  - `calinski_harabasz_score` (выше – лучше);
- аккуратную **визуализацию** результатов:
  - PCA(2D) обязательно,
  - t-SNE (опционально, с правильными оговорками);
- оформление результата в виде ноутбука + короткого отчёта + артефактов (как в предыдущих ДЗ).

---

## 2. Задание

### 2.1. Структура для HW07 (обязательно)

1) В корне репозитория должна быть папка `homeworks/` (создать, если её ещё нет).  
2) Внутри `homeworks/` создать папку `HW07/`.  
3) В папке `homeworks/HW07/` создать:

- основной ноутбук: `HW07.ipynb`
- отчёт: `report.md`
- папку с данными: `data/`
- папку для артефактов: `artifacts/`
  - рекомендуется внутри `artifacts/` завести подпапку `figures/` для графиков

> Имена папок и файлов должны быть **строго такими**, как указано (регистр важен).

---

### 2.2. Учебные датасеты S07 (выбрать 3 из 4)

Для HW07 предоставлены **4 синтетических CSV-файла**. Нужно выбрать **любые 3** (четвёртый – опционально).

Положите выбранные CSV в `homeworks/HW07/data/`.

Файлы:

- `S07-hw-dataset-01.csv`  
  Числовые признаки в разных шкалах + шумовые признаки. Без масштабирования результаты обычно "едут".

- `S07-hw-dataset-02.csv`  
  Нелинейная структура + выбросы + лишний шумовой признак. Хорошо демонстрирует, где KMeans проигрывает.

- `S07-hw-dataset-03.csv`  
  Кластеры разной плотности + фоновый шум. Часто провоцирует ошибки выбора `eps` для DBSCAN.

- `S07-hw-dataset-04.csv`  
  Высокая размерность + 2 категориальных признака + пропуски в числовых. Требует аккуратного препроцессинга.

Во всех CSV:

- есть колонка `sample_id` (это **не** признак, используйте только как идентификатор);
- истинных меток кластеров **нет**.

Требование к путям: в ноутбуке используйте **относительные пути** (без абсолютных путей к домашним каталогам).

---

### 2.3. Содержание ноутбука `HW07.ipynb` (основная часть)

В ноутбуке `homeworks/HW07/HW07.ipynb` необходимо выполнить следующие шаги.

#### 2.3.1. Загрузка данных и первичный анализ (для каждого датасета)

Для **каждого** из 3 выбранных CSV:

1) Загрузить CSV в `pandas.DataFrame`.  
2) Зафиксировать минимум:
   - `head()`, `info()`, базовые статистики (`describe()` или аналог);
   - проверка пропусков (кол-во/доли);
   - типы признаков (числовые / категориальные).  
3) Определить:
   - `X` – признаки (все столбцы, кроме `sample_id`);
   - `sample_id` хранить отдельно (для сохранения результатов).

#### 2.3.2. Препроцессинг (обязательно)

Для каждого датасета оформите препроцессинг **явно** и применяйте его одинаково ко всем моделям данного датасета.

Минимум:

- масштабирование числовых признаков: `StandardScaler`;
- если есть пропуски – обработать (`SimpleImputer`);
- если есть категориальные признаки (dataset-04) – один из вариантов:
  - закодировать (`OneHotEncoder(handle_unknown="ignore")`), **или**
  - осознанно исключить категориальные признаки и объяснить почему (в отчёте).

Рекомендация: использовать `Pipeline`/`ColumnTransformer` (не обязательно идеально, но логика должна быть понятной).

#### 2.3.3. Модели недели 7 (для каждого датасета – минимум 2 алгоритма)

Для **каждого датасета** сравните минимум **2** алгоритма:

1) **KMeans** (обязательно):
   - подобрать `k` в разумном диапазоне (например, 2…20);
   - фиксировать `random_state` и `n_init`;
   - показать хотя бы один график "метрика vs k" (например, silhouette vs k).

2) **Один алгоритм на выбор** (обязательно):
   - `DBSCAN` (подбор `eps` и `min_samples`), **или**
   - `AgglomerativeClustering` (подбор `k` + выбор `linkage`, хотя бы 2 варианта).

> Можно делать 3-й алгоритм (приветствуется), но он не обязателен.

#### 2.3.4. Метрики качества (обязательно)

Для каждого датасета и каждого сравниваемого алгоритма посчитать:

- `silhouette_score`
- `davies_bouldin_score`
- `calinski_harabasz_score`

Важно для DBSCAN:

- учесть шум (`label = -1`):
  - явно вывести долю шума,
  - метрики считать либо на non-noise точках (и это указать), либо честно объяснить иной выбор.

#### 2.3.5. Визуализация (обязательно)

Для каждого датасета:

- PCA(2D) scatter с раскраской по полученным кластерам (для **лучшего** решения по датасету);
- минимум один дополнительный график по ходу подбора параметров (например, silhouette vs k или silhouette vs eps).

t-SNE – опционально:

- если делаете, фиксируйте `random_state` и коротко поясните, как правильно интерпретировать t-SNE (это визуализация локальной структуры, а не "доказательство качества").

#### 2.3.6. Устойчивость (обязательно, но только для одного датасета)

Выберите **один** из ваших датасетов и проведите мини-проверку устойчивости:

- для KMeans: 5 запусков с разными `random_state` (или 5 разными подвыборками) и оценка похожести разбиений (например, ARI между результатами), **или**
- любая другая аккуратная проверка устойчивости (кратко описать и обосновать).

#### 2.3.7. Итог по каждому датасету (обязательно)

Для каждого датасета в конце блока:

- выбрать "лучший" метод/настройку (не обязательно тот, где максимум silhouette – главное, чтобы выбор был объяснён);
- написать 5-10 строк: что получилось, где были сложности (шкалы/выбросы/плотность/пропуски), почему выбранный метод уместен.

---

### 2.4. Артефакты эксперимента (обязательно)

В папке `homeworks/HW07/artifacts/` должны быть:

- `metrics_summary.json` – сводка метрик по датасетам и моделям (silhouette/DB/CH + доля шума для DBSCAN);
- `best_configs.json` – какие параметры выбраны как "лучшие" для каждого датасета (и каким критерием);
- `labels/` – CSV-файлы с присвоенными кластерами для **лучшего** решения на каждом датасете, например:
  - `labels_ hw07_ds1.csv`, `labels_hw07_ds2.csv`, ...
  Формат: `sample_id,cluster_label` (для DBSCAN кластер `-1` допустим);
- `figures/` – минимум 6 изображений:
  - по 1 PCA(2D) scatter на каждый из 3 датасетов (итого минимум 3),
  - и ещё минимум 3 графика "подбор параметров / метрики" (например, silhouette vs k/eps, сравнение linkage и т.п.).

> Формат артефактов (json/csv/png) можно выбирать свободно, главное – чтобы их можно было открыть и понять без запуска ноутбука.

---

### 2.5. Отчёт `report.md` (обязательно)

1) В материалах семинара будет шаблон: `S07-hw-report-template.md`.  
2) Нужно создать файл `homeworks/HW07/report.md` и заполнить его **по шаблону**.

Важно:

- не меняйте названия разделов (заголовков) в отчёте;
- вставляйте результаты и выводы в соответствующие секции.

---

## 3. Требования к структуре и именованию (итог)

К дедлайну в репозитории должно быть:

- `homeworks/HW07/HW07.ipynb`
- `homeworks/HW07/report.md`
- `homeworks/HW07/data/` (3 выбранных CSV)
- `homeworks/HW07/artifacts/` (см. состав выше)

Требования:

- названия папок и файлов - строго как указано;
- путь к CSV - относительный;
- ноутбук выполняется **без ошибок** при последовательном запуске всех ячеек;
- результаты эксперимента оформлены: метрики, сравнение алгоритмов, визуализации, выводы.

---

## 4. Критерии зачёта

HW07 считается зачтённым, если:

1) Соблюдена структура `homeworks/HW07/` и нейминг файлов.

2) В `HW07.ipynb` есть (для **каждого** из 3 датасетов):

   - загрузка выбранного CSV;
   - базовый EDA (тип/пропуски/описание признаков);
   - явный препроцессинг (scaling обязательно; пропуски/категориальные – если есть);
   - сравнение минимум 2 алгоритмов (KMeans + (DBSCAN или Agglomerative));
   - расчёт внутренних метрик (silhouette/DB/CH) и их интерпретация;
   - PCA(2D) визуализация для лучшего решения;
   - текстовый вывод по датасету.

3) Есть проверка устойчивости (хотя бы на одном датасете).

4) В `artifacts/` лежат требуемые файлы и минимум 6 графиков в `figures/`.

5) Заполнен `report.md` по шаблону.

---

## 5. Опциональная часть (для желающих)

Не обязательна для зачёта, но приветствуется:

- t-SNE для 1-2 датасетов (с фиксированным `random_state` и аккуратной интерпретацией);
- сравнение времени выполнения (fit/predict) разных подходов на больших датасетах;
- более аккуратный подбор параметров DBSCAN (например, k-distance plot как эвристика);
- сравнение результатов при разных вариантах препроцессинга (например, с PCA и без).

---

## 6. Сроки и порядок сдачи

- Работа выполняется **индивидуально**.
- Дедлайн объявляется преподавателем отдельно.
- Факт сдачи: к дедлайну в репозитории есть `homeworks/HW07/` со всеми файлами и корректно выполненным ноутбуком.


In [1]:
# Импорты: только стандартные библиотеки + scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

#### 2.3.1. Загрузка данных и первичный анализ (для каждого датасета)

In [8]:
# Загрузка датасетов
d1 = pd.read_csv("data/S07-hw-dataset-01.csv")
print("Shape:", d1.shape)
d1.head()

Shape: (12000, 9)


,sample_id,f01,f02,f03,f04,f05,f06,f07,f08
0,0,-0.536647,-69.812900,-0.002657,71.743147,-11.396498,-12.291287,-6.836847,-0.504094
1,1,15.230731,52.727216,-1.273634,-104.123302,11.589643,34.316967,-49.468873,0.390356
2,2,18.542693,77.317150,-1.321686,-111.946636,10.254346,25.892951,44.595250,0.325893
3,3,-12.538905,-41.709458,0.146474,16.322124,1.391137,2.014316,-39.930582,0.139297
4,4,-6.903056,61.833444,-0.022466,-42.631335,3.107154,-5.471054,7.001149,0.131213


In [9]:
d1.describe()

,sample_id,f01,f02,f03,f04,f05,f06,f07,f08
count,12000.00000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,5999.50000,-2.424716,19.107804,-0.222063,-8.284501,-0.190717,0.962972,0.033724,0.007638
std,3464.24595,11.014315,60.790338,0.500630,59.269838,7.026435,14.794713,59.541782,0.607053
min,0.00000,-19.912573,-92.892652,-1.590979,-134.303679,-11.869169,-20.521164,-215.098834,-2.633469
25%,2999.75000,-9.472623,-40.282955,-0.125145,-48.345007,-5.132473,-8.807706,-39.900520,-0.401483
50%,5999.50000,-6.869404,54.069335,-0.031753,16.211728,0.444730,-6.134169,-0.578494,0.005306
75%,8999.25000,0.523841,70.280739,0.054980,28.067178,3.942368,2.334426,39.719821,0.410132
max,11999.00000,24.403381,112.229523,0.512277,75.088604,13.717091,41.452857,213.381767,2.490745


In [18]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sample_id  12000 non-null  int64  
 1   f01        12000 non-null  float64
 2   f02        12000 non-null  float64
 3   f03        12000 non-null  float64
 4   f04        12000 non-null  float64
 5   f05        12000 non-null  float64
 6   f06        12000 non-null  float64
 7   f07        12000 non-null  float64
 8   f08        12000 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 843.9 KB


In [17]:
d1.isna().sum()

sample_id    0
f01          0
f02          0
f03          0
f04          0
f05          0
f06          0
f07          0
f08          0
dtype: int64

In [5]:
d2 = pd.read_csv("data/S07-hw-dataset-02.csv")
print("Shape:", d2.shape)
d2.head()

Shape: (8000, 4)


,sample_id,x1,x2,z_noise
0,0,0.098849,-1.846034,21.288122
1,1,-1.024516,1.829616,6.072952
2,2,-1.094178,-0.158545,-18.938342
3,3,-1.612808,-1.565844,-11.629462
4,4,1.659901,-2.133292,1.895472


In [11]:
d2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sample_id  8000 non-null   int64  
 1   x1         8000 non-null   float64
 2   x2         8000 non-null   float64
 3   z_noise    8000 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 250.1 KB


In [19]:
d2.isna().sum() 

sample_id    0
x1           0
x2           0
z_noise      0
dtype: int64

In [12]:
d2.describe()

,sample_id,x1,x2,z_noise
count,8000.00000,8000.000000,8000.000000,8000.000000
mean,3999.50000,0.478867,0.241112,0.110454
std,2309.54541,0.955138,0.663195,8.097716
min,0.00000,-2.487352,-2.499237,-34.056074
25%,1999.75000,-0.116516,-0.242357,-5.392210
50%,3999.50000,0.490658,0.241092,0.132470
75%,5999.25000,1.085263,0.726526,5.655605
max,7999.00000,2.987555,2.995553,29.460076


In [6]:
d3 = pd.read_csv("data/S07-hw-dataset-03.csv")
print("Shape:", d3.shape)
d3.head()

Shape: (15000, 5)


,sample_id,x1,x2,f_corr,f_noise
0,0,-2.710470,4.997107,-1.015703,0.718508
1,1,8.730238,-8.787416,3.953063,-1.105349
2,2,-1.079600,-2.558708,0.976628,-3.605776
3,3,6.854042,1.560181,1.760614,-1.230946
4,4,9.963812,-8.869921,2.966583,0.915899


In [20]:
d3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sample_id  15000 non-null  int64  
 1   x1         15000 non-null  float64
 2   x2         15000 non-null  float64
 3   f_corr     15000 non-null  float64
 4   f_noise    15000 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 586.1 KB


In [21]:
d3.describe()

,sample_id,x1,x2,f_corr,f_noise
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7499.500000,1.246296,1.033764,0.212776,-0.027067
std,4330.271354,4.592421,4.710791,1.530017,2.506375
min,0.000000,-9.995585,-9.980853,-5.212038,-8.785884
25%,3749.750000,-1.782144,-2.666393,-0.966224,-1.731128
50%,7499.500000,0.664226,1.831257,0.296508,-0.052391
75%,11249.250000,4.435671,4.969630,1.390273,1.673831
max,14999.000000,16.207863,14.271153,5.795876,11.266865


In [24]:
d3.isna().sum() 

sample_id    0
x1           0
x2           0
f_corr       0
f_noise      0
dtype: int64

In [31]:
sample_id = d1["sample_id"].copy()
X1 = d1.drop(columns=["sample_id"])
print("X1 shape:", X1.shape)
print("sample_id shape:", sample_id.shape)
X1.head()

X1 shape: (12000, 8)
sample_id shape: (12000,)


,f01,f02,f03,f04,f05,f06,f07,f08
0,-0.536647,-69.812900,-0.002657,71.743147,-11.396498,-12.291287,-6.836847,-0.504094
1,15.230731,52.727216,-1.273634,-104.123302,11.589643,34.316967,-49.468873,0.390356
2,18.542693,77.317150,-1.321686,-111.946636,10.254346,25.892951,44.595250,0.325893
3,-12.538905,-41.709458,0.146474,16.322124,1.391137,2.014316,-39.930582,0.139297
4,-6.903056,61.833444,-0.022466,-42.631335,3.107154,-5.471054,7.001149,0.131213


In [32]:
sample_id = d2["sample_id"].copy()
X2 = d2.drop(columns=["sample_id"])
print("X2 shape:", X2.shape)
print("sample_id shape:", sample_id.shape)
X2.head()

X2 shape: (8000, 3)
sample_id shape: (8000,)


,x1,x2,z_noise
0,0.098849,-1.846034,21.288122
1,-1.024516,1.829616,6.072952
2,-1.094178,-0.158545,-18.938342
3,-1.612808,-1.565844,-11.629462
4,1.659901,-2.133292,1.895472


In [33]:
sample_id = d3["sample_id"].copy()
X3 = d3.drop(columns=["sample_id"])
print("X3 shape:", X3.shape)
print("sample_id shape:", sample_id.shape)
X3.head()

X3 shape: (15000, 4)
sample_id shape: (15000,)


,x1,x2,f_corr,f_noise
0,-2.710470,4.997107,-1.015703,0.718508
1,8.730238,-8.787416,3.953063,-1.105349
2,-1.079600,-2.558708,0.976628,-3.605776
3,6.854042,1.560181,1.760614,-1.230946
4,9.963812,-8.869921,2.966583,0.915899


#### 2.3.2. Препроцессинг (обязательно)

#### 2.3.3. Модели недели 7 (для каждого датасета – минимум 2 алгоритма)

#### 2.3.4. Метрики качества (обязательно)

#### 2.3.5. Визуализация (обязательно)

#### 2.3.6. Устойчивость (обязательно, но только для одного датасета)

#### 2.3.7. Итог по каждому датасету (обязательно)

### 2.4. Артефакты эксперимента (обязательно)